# Vector and matrix

#### Để tìm hiểu rõ hơn về các khái niệm liên quan tới đại số tuyến tính, có thể tham khảo ở link bên dưới:
Authors: Vũ Hữu Tiệp và nhóm dịch sách Machine Learning cơ bản. <br>
Kiến thức liên quan: Chương I <br>
Link tham khảo: [Ebook sách Machine Learning cơ bản](https://github.com/tiepvupsu/tiepvupsu.github.io/blob/master/ML_math.pdf)<br>

## Một số ứng dụng của vector và matrix

### Lật ảnh đối xứng qua trục ngang

<img align="left" src="vector_matrix_apli_1.png" >


Xét một điểm M được biểu diễn bởi vector x trong không gian hai chiều thì $x\in R^{2}$ và nếu ta sử dụng ma trận $A=\begin{vmatrix}
1 & 0 \\ 
0 & -1
\end{vmatrix}$. Phép biến đổi từ vector x thành vector b thông qua phép biến đổi $Ax=b$ thì sẽ sinh ra điểm $M^{'}$ đối xứng với điểm $M$ qua trục hoành. Ứng dụng tính chất cơ bản này ta có thể đảo ngược một ảnh bất kỳ bằng cách sử dụng ma trận $A$ để sinh ra các điểm đối xứng với từng pixel qua trục hoành. <br>

<br>
    Ma trận tổng qua hơn ma trận A là ma trận $R=\begin{vmatrix}
cos\theta & -sin\theta \\ 
sin\theta & cos\theta
\end{vmatrix}$ dùng để xoay ảnh theo một góc $\theta$ bất kỳ. <br>

Trong OpenCV và Numpy hỗ trợ một số hàm cơ bản để thực hiện việc xoay ảnh, hay lật một ảnh:<br>
   1. Rotate image with OpenCV: `cv2.rotate()` 
   2. Flip image with OpenCV: `cv2.flip()`
   3. Rotate image with NumPy: `np.rot90()`
   4. Flip image with NumPy: `np.flip()`
  
Dưới đây là code và kết quả minh họa:

In [3]:
import cv2

img = cv2.imread('lena.jpg')
print(type(img))
# <class 'numpy.ndarray'>

print(img.shape)
# (225, 400, 3)

img_rotate_90_clockwise = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
cv2.imwrite('lena_cv_rotate_90_clockwise.jpg', img_rotate_90_clockwise)
# True

img_rotate_90_counterclockwise = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
cv2.imwrite('lena_cv_rotate_90_counterclockwise.jpg', img_rotate_90_counterclockwise)
# True

img_rotate_180 = cv2.rotate(img, cv2.ROTATE_180)
cv2.imwrite('lena_cv_rotate_180.jpg', img_rotate_180)


<class 'numpy.ndarray'>
(225, 400, 3)


Result:<br>
    <img align="left" src="lena_cv_rotate_90_clockwise.jpg" >
    <img align="left" src="lena_cv_rotate_90_counterclockwise.jpg" >
    <img align="left" src="lena_cv_rotate_180.jpg" >

#### Ta sẽ sử dụng thư viện `imutils` để xoay ảnh với một góc bất kỳ

**Syntax:**<br>
$\hspace{3cm}$        `imutils.rotate(image, angle=angle)`

In [8]:
!pip install imutils
import cv2
import imutils

image = cv2.imread(r"lena.jpg")
 
rot = imutils.rotate(image, angle=45)
cv2.imshow("Rotated", rot)
cv2.waitKey(0)

Using legacy 'setup.py install' for imutils, since package 'wheel' is not installed.
    Running setup.py install for imutils: started
    Running setup.py install for imutils: finished with status 'done'


-1

# Background Substraction

Giải thuật Background Subtraction (tạm dịch: trừ nền) là giải thuật mà ta sẽ cần có 2 ảnh, một ảnh nền và một ảnh có đối tượng, ta lấy 2 ảnh đó để trừ nhau. Mục đích là bằng cách loại bỏ nền ta sẽ giữ lại được đối tượng có trên ảnh.<br>

Giải thuật này phù hợp cho các trường hợp xử lý ảnh khi đã biết trước, hoặc tính toán được ảnh nền (background) từ camera. Chi phí chạy giải thuật nhỏ nên thường được áp dụng trên các bo nhúng.

### Hiện thực giải thuật Background Subtraction trên ảnh màu
#### Cách làm thực ra chỉ cần một bước duy nhất là lấy 2 ảnh trừ nhau, từ đó chuẩn hóa để tạo mặt nạ cho foreground. Nhưng các bước sau đây bổ sung một số bước để trích xuất đối tượng tốt hơn. <br>
1. Làm mờ ảnh background lẫn foreground bằng bộ lọc làm mờ Gaussian.
2. Lấy ảnh foreground trừ đi ảnh background.
3. Do chúng ta vừa mới trừ hai ảnh cho nhau mà giá trị pixel thì nằm trong khoảng [0;255] nên sau khi trừ giá trị là [-255;255]. Ta phải chuẩn hóa để đưa mask về chuẩn $0\rightarrow 1$ hoặc $0\rightarrow 255$.
4. Do ảnh foreground và background đưa vào đều là ảnh màu, chúng có đến 3 kênh lận. Vì vậy muốn còn 1 kênh thì có thể trung bình các giá trị của 3 kênh màu này để còn giữ duy nhất một kênh cho mask. Miền giá trị các pixel trên mặt nạ là 0->255 (cho dễ visualize in thành ảnh)
5. Nhị phân hóa mask ta vừa thu được. Bản chất của mỗi pixel trên mặt nạ thể hiện độ khác biệt trên từng pixel tương ứng giữa ảnh background và ảnh foreground! Nếu chúng khác biệt càng lớn thì có khả năng pixel tại đấy có đối tượng hoặc biến đổi về mức sáng. Do đó ta cần phải tự đặt cho mình một siêu tham số để nhị phân hóa ảnh. Giá trị của siêu tham số này có ảnh hưởng quan trọng đến chất lượng kết quả ảnh đầu ra cuối cùng đấy. Thử chỉnh nhè nhẹ giá trị của nó sẽ nhận thấy sự khác biệt.
6. Sau khi có ảnh, làm mờ bằng bộ lọc Median Blur. Vì quan sát trên ảnh mask (lúc chưa áp dụng làm mờ với Median filter) kết quả sẽ thấy rằng có nhiễu muối tiêu rất nhiều (lốm đốm các pixel trắng giữa rằng đen hoặc pixel đen giữa rừng trắng), mà Median Filter là thằng chuyên trị nhiễu muối tiêu.

Các thao tác còn lại trong đoạn code dưới là cố nhét ảnh kết quả vào ma trận 4 kênh để tạo ảnh PNG. Ảnh màu thông thường (nén bằng JPEG) chỉ có 3 kênh, ảnh PNG không nén mà lại có thêm transparent nữa thì cấu trúc lưu trữ bên dưới của nó nhét thêm một kênh thứ 4 nữa. Kênh thứ 4 này mang giá trị biểu diễn mức độ trong suốt của pixel đó (0 = trong suốt hoàn toàn: transparent, 255 = đặc hoàn toàn: opaque, giá trị ở giữa 0->255 = mờ mờ ảo ảo)

In [9]:
"""
Author : Minh Nguyen
Giải thuật Background Subtraction trên ảnh màu
Ref: https://minhng.info/tutorials/giai-thuat-background-subtraction-anh-mau.html
"""
import cv2
import numpy as np

FOREGROUND_IMG = 'foreground.jpg'
BACKGROUND_IMG = 'background.jpg'

def blur_color_img(img, kernel_width=5, kernel_height=5, sigma_x=2, sigma_y=2):
    img = np.copy(img) # we don't modify the original image
    img[:,:,0] = cv2.GaussianBlur(img[:,:,0], ksize=(kernel_width, kernel_height), sigmaX=sigma_x, sigmaY=sigma_y)
    img[:,:,1] = cv2.GaussianBlur(img[:,:,1], ksize=(kernel_width, kernel_height), sigmaX=sigma_x, sigmaY=sigma_y)
    img[:,:,2] = cv2.GaussianBlur(img[:,:,2], ksize=(kernel_width, kernel_height), sigmaX=sigma_x, sigmaY=sigma_y)
    return img   

def background_subtraction(fg_img, bg_img, diff_threshold=30):
    fg_img = blur_color_img(fg_img, 7, 7, 4, 4)
    bg_img = blur_color_img(bg_img, 7, 7, 4, 4)
    mask = fg_img - bg_img
    mask = np.abs(mask)
    mask = np.mean(mask, axis=2, keepdims=False)
    mask[mask<diff_threshold] = 0
    mask[mask>=diff_threshold] = 255
    mask = mask.astype(np.uint8)
    mask = cv2.medianBlur(mask, 7)
    return mask
    
def main(foreground_img, background_img):
    fg_img = cv2.imread(foreground_img) # [h, w, 3]
    bg_img = cv2.imread(background_img) # [h, w, 3]
    mask = background_subtraction(fg_img, bg_img)
    new_fg = np.zeros([fg_img.shape[0], fg_img.shape[1], 4]) # png image --> has 4-dims instead of 3-dims like color image
    new_fg[:,:,:3] = fg_img
    new_fg[:,:,3] = mask
    cv2.imwrite('mask.jpg', mask)
    cv2.imwrite('captain_america.png', new_fg)
    
if __name__ == "__main__":
    print('Running Background Subtraction for: %s and %s' % (FOREGROUND_IMG, BACKGROUND_IMG))
    main(foreground_img=FOREGROUND_IMG, background_img=BACKGROUND_IMG)

Running Background Subtraction for: foreground.jpg and background.jpg


Input:<br>
<img align="left" src="background.jpg" >
<img align="left" src="foreground.jpg" >

Result: <br>
$\hspace{3cm}$ <img align="left" src="captain_america.png" >

Tham khảo:
1. [Rotation matrix](https://en.wikipedia.org/wiki/Rotation_matrix)
2. [Ebook sách Machine Learning cơ bản](https://github.com/tiepvupsu/tiepvupsu.github.io/blob/master/ML_math.pdf)
3. [OpenCV, NumPy: Rotate and flip image](https://note.nkmk.me/en/python-opencv-numpy-rotate-flip/)
4. [2 ways to rotate an image by an angle in Python](https://www.askpython.com/python/examples/rotate-an-image-by-an-angle-in-python)
5. [Minh Nguyễn, _Giải thuật Background Subtraction trên ảnh màu_, Visited: 27/10:](https://minhng.info/tutorials/giai-thuat-background-subtraction-anh-mau.html)